In [58]:
import psycopg2
import os
import pandas as pd
import ast
import re

from dotenv import load_dotenv
load_dotenv()

KEY_POSTGRES = os.getenv("KEY_POSTGRES")

In [64]:
DF_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\preparedInstitutionsTestDatabaseAccess.csv


In [65]:
df = pd.read_csv(DF_input)

In [66]:
s = df.iloc[2]["NER_BERT"]

In [67]:
df.iloc[2]["Institutions_man"]

"['Instituto de Investigação e Inovação em Saúde', 'University of Porto', 'University of British Columbia', 'BC Cancer', 'Institute of Molecular Pathology and Immunology']"

In [68]:
cleaned = re.sub(r"np\.float\d+\((.*?)\)", r"\1", s)
cleaned

"[{'entity_group': 'INS', 'score': 0.99988544, 'word': 'University of British Columbia', 'start': 32, 'end': 62}, {'entity_group': 'SUBC', 'score': 0.99957687, 'word': 'Vancouver', 'start': 64, 'end': 73}, {'entity_group': 'COU', 'score': 0.9991843, 'word': 'Canada', 'start': 75, 'end': 81}, {'entity_group': 'INS', 'score': 0.9996547, 'word': 'Hereditary Cancer', 'start': 112, 'end': 129}, {'entity_group': 'INS', 'score': 0.9165015, 'word': 'BC Cancer', 'start': 131, 'end': 140}, {'entity_group': 'INS', 'score': 0.9998889, 'word': 'University of British Columbia', 'start': 196, 'end': 226}, {'entity_group': 'SUBC', 'score': 0.99966955, 'word': 'Vancouver', 'start': 228, 'end': 237}, {'entity_group': 'COU', 'score': 0.9899244, 'word': 'Canada', 'start': 239, 'end': 245}, {'entity_group': 'INS', 'score': 0.9998265, 'word': 'Instituto de Investigação e Inovação em Saúde', 'start': 253, 'end': 298}, {'entity_group': 'INS', 'score': 0.9998478, 'word': 'Institute of Molecular Pathology and I

In [91]:
query = ast.literal_eval(cleaned)[6]["word"]
query

'Vancouver'

In [92]:
connection = psycopg2.connect(database="ror_institutions", user="postgres", password=KEY_POSTGRES, host="localhost", port=5432)
cursor = connection.cursor()
cursor.execute("SELECT DISTINCT ON (institution_id) name FROM public.institution_names WHERE name = %s;", (query,))
record = cursor.fetchall()
record

[]

In [53]:
ast.literal_eval(cleaned)[0]

{'entity_group': 'INS',
 'score': 0.99988544,
 'word': 'University of British Columbia',
 'start': 32,
 'end': 62}

In [102]:
df['Institutions_db'] = [[] for _ in range(len(df))]
df['Countries_db'] = [[] for _ in range(len(df))]

for row in range(len(df)):
    cleaned = re.sub(r"np\.float\d+\((.*?)\)", r"\1", df.iloc[row]["NER_BERT"])
    for NER_result in ast.literal_eval(cleaned):
        if NER_result['entity_group'] == 'INS':
            query = NER_result['word']
            connection = psycopg2.connect(database="ror_institutions", user="postgres", password=KEY_POSTGRES, host="localhost", port=5432)
            cursor = connection.cursor()
            cursor.execute(f"SELECT * FROM institution_names WHERE name = '{query}';")
            record = cursor.fetchall()
            institutions_db = []
            for el in record:
                institutions_db.append(el[3])
            institutions_db = list(set(institutions_db))
            if len(institutions_db) == 1:
                df.iloc[row]['Institutions_db'].append(institutions_db)
                i = 0
                while i == 0:
                    for el in record:
                        if institutions_db[0] in el:
                            new_query = el[2]
                            connection = psycopg2.connect(database="ror_institutions", user="postgres", password=KEY_POSTGRES, host="localhost", port=5432)
                            cursor = connection.cursor()
                            cursor.execute(f"SELECT country_name FROM institution_locations WHERE country_name = '{new_query}';")
                            new_record = cursor.fetchall()
                            if len(new_record) > 0:
                                df.iloc[row]['Countries_db'].append(new_record[0][5])
                                i += 1

OperationalError: connection to server at "localhost" (::1), port 5432 failed: server sent an error response during SSL exchange
